# Chapter 05 - Exercises

### Exercise 5.6

**Q**

What is the equation analogous to (5.6) for action values Q(s, a) instead of state values V (s), again given returns generated using b?

**A**

The equation 5.6 is:

\begin{align*}
V(s) \stackrel{.}{=} \frac{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1}}
\end{align*}

And:

\begin{align*}
\rho_{t:T(t)-1} \stackrel{.}{=} \frac{\prod_{k=t}^{T-1} \pi(A_k | S_k) p(S_{k+1} | S_k, A_k)}{\prod_{k=t}^{T-1} b(A_k | S_k) p(S_{k+1} | S_k, A_k)} = \frac{\prod_{k=t}^{T-1} \pi(A_k | S_k)}{\prod_{k=t}^{T-1} b(A_k | S_k)}
\end{align*}

For $Q(s, a)$, we have $A_t=a$, so:

\begin{align*}
Q(s, a) &\stackrel{.}{=} \frac{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:t} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \rho_{t:t} \rho_{t+1:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(A_t | S_t)}{b(A_t | S_t)} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(A_t | S_t)}{b(A_t | S_t)} \rho_{t+1:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(a | s)}{b(a | s)} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \frac{\pi(a | s)}{b(a | s)} \rho_{t+1:T(t)-1}} \\
&= \frac{\frac{\pi(a | s)}{b(a | s)} \sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1} G_t}{\frac{\pi(a | s)}{b(a | s)} \sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1}} \\
&= \frac{\sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s, a)} \rho_{t+1:T(t)-1}}
\end{align*}

In which $\mathcal{T}(s, a)$ are the occurences (or first occurence in the episode for first-visit Monte Carlo) of the state s and action a across all episodes.

Because $A_t=a$, the first term in the product that generate $\rho_{t:T(t)-1}$ is a constant, and can be isolated outside the sum. 

In a weighted importance sampling (which is the case above), the constant is present both in the numerator and denominator and cancel each other, that is, the importance-sampling ratio, $\rho$, can consider only the times from $t+1$ onwards in a weighted importance sampling for $Q(s, a)$.



### Exercise 5.7

**Q**

In learning curves such as those shown in Figure 5.3 error generally decreases with training, as indeed happened for the ordinary importance-sampling method. But for the weighted importance-sampling method error first increased and then decreased. Why do you think this happened?

**A**

### Exercise 5.8

**Q**

The results with Example 5.5 and shown in Figure 5.4 used a first-visit MC method. Suppose that instead an every-visit MC method was used on the same problem. Would the variance of the estimator still be infinite? Why or why not?

**A**

Yes. The increase in value is exponential, while the every-visit MC would simply average the results of an episode accross the occurences, so the total would still diverges into infinite.

Mathematically, while the case for first visits can be defined as:

\begin{align*}
\mathbb{E}[X^2] = \mathbb{E}_b\left[\left(\prod_{t=0}^{T-1} \frac{\pi(A_t | S_t)}{b(A_t | S_t)} G_0\right)^2\right] = \mathbb{E}_b\left[\left(\prod_{t=0}^{T-1} \frac{\pi(A_t | S_t)}{b(A_t | S_t)}\right)^2\right] = 0.1 \sum_{k=0}^{\infty} 0.9^k \cdot 2^k \cdot 2 = 0.2 \sum_{k=0}^{\infty} 1.8^k = \infty
\end{align*}

in which 0.1 is due to the probability of the left action ending the episode (10%), which will always happen at the end (only once), 0.9 is due to the probability of left action returning back to the same state (90%), which can happen indefinitelly, and $2^k$ is due to the product of choosing the left action k times under policy b, with the ratio of the policies of choosing left k times squared $0.5^k \cdot \left(\frac{1}{0.5}\right)^{2k} = \left(\frac{1}{0.5}\right)^k = 2^k$.

For an every-visit MC, for a case in which the left action returned back n times, and in the n+1 time it ended the episode, instead of considering only the first occurence, $0.1 \cdot 0.9^n$, it will consider all cases and calculate the average, $\frac{\sum_{i=0}^n 0.1 \cdot 0.9^i}{n+1}$ (it's n+1 time-steps: n time-steps returning to the same position, and one last ending the episode), so the variance is:

\begin{align*}
\mathbb{E}[X^2] &= \mathbb{E}_b\left[\left(\prod_{t=0}^{T-1} \frac{\pi(A_t | S_t)}{b(A_t | S_t)} G_0\right)^2\right] \\
&= \mathbb{E}_b\left[\left(\prod_{t=0}^{T-1} \frac{\pi(A_t | S_t)}{b(A_t | S_t)}\right)^2\right] \\
&= 0.1 \sum_{k=0}^{\infty} \frac{\sum_{i=0}^k 0.9^i \cdot 2^i}{k+1} \cdot 2 \\
&= 0.2 \sum_{k=0}^{\infty} \frac{\sum_{i=0}^k 1.8^i}{k} \\
&= 0.2 \sum_{k=0}^{\infty} \frac{1.8^k}{k} + 0.2 \sum_{k=0}^{\infty} \frac{\sum_{i=0}^{k-1} 1.8^i}{k} \\
&= \infty
\end{align*}

The above equation is infinite because the first term of the sum is infinite, and the second is positive:

\begin{align*}
\operatorname*{lim}_{x \to \infty}{0.2 \sum_{k=0}^x \frac{1.8^k}{k}} > 0.2 \operatorname*{lim}_{x \to \infty}{\frac{1.8^x}{x}} = \infty
\end{align*}

In the above equation, the numerator grows exponentially, while the denominator grows linearly, both toward $\infty$, so the division is $\infty$. We can apply L'Hôpital's rule since both terms grow to inifity:

\begin{align*}
\operatorname*{lim}_{x \to \infty}{\frac{1.8^x}{x}} &= \operatorname*{lim}_{x \to \infty}{\frac{\frac{d(1.8^x)}{dx}}{\frac{dx}{dx}}} \\
&= \operatorname*{lim}_{x \to \infty}{\frac{1.8^x ln(1.8)}{1}} \\
&= ln(1.8) \operatorname*{lim}_{x \to \infty}{1.8^x} \\
&= \operatorname*{lim}_{x \to \infty}{1.8^x} \\
& = \infty
\end{align*}

So, the variance of the estimator would still be infinite.